In [15]:
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.decomposition import PCA


In [2]:
row_numbers = np.arange(5,16,1)
print(row_numbers)

[ 5  6  7  8  9 10 11 12 13 14 15]


In [3]:
res = pd.DataFrame(columns=('column_selected','precision','recall','finding_rate','cross_val'))

def load_one_version_method_matrix(matrix_path):
    print(matrix_path)
    return pd.read_csv(matrix_path,header=None, names = ['column_selected','precision','recall','finding_rate','cross_val'] )

filePath = "rows_selected/"
for row_number in row_numbers:
    selected_row_number = "Rows_selected_"+str(row_number)+"_result.csv"
    result = load_one_version_method_matrix(filePath+selected_row_number)
    sorted_result = result.sort_values(by="finding_rate" , ascending=False) 
    res = res.append(sorted_result[0:5], ignore_index=True)


rows_selected/Rows_selected_5_result.csv
rows_selected/Rows_selected_6_result.csv
rows_selected/Rows_selected_7_result.csv
rows_selected/Rows_selected_8_result.csv
rows_selected/Rows_selected_9_result.csv
rows_selected/Rows_selected_10_result.csv
rows_selected/Rows_selected_11_result.csv
rows_selected/Rows_selected_12_result.csv
rows_selected/Rows_selected_13_result.csv
rows_selected/Rows_selected_14_result.csv
rows_selected/Rows_selected_15_result.csv


In [12]:
res

,column_selected,precision,recall,finding_rate,cross_val
0,"(0, 1, 2, 5, 13)",0.978102,0.905405,"(0.5862068965517241, 34)",[0.78873239 0.81690141 0.82857143 0.77142857 0...
1,"(4, 6, 7, 10, 14)",1.000000,0.844595,"(0.5689655172413793, 33)",[0.81690141 0.81690141 0.81428571 0.82857143 0...
2,"(0, 1, 3, 4, 13)",0.992701,0.918919,"(0.5689655172413793, 33)",[0.8028169 0.81690141 0.8 0.8 0...
3,"(0, 1, 7, 9, 10)",0.985507,0.918919,"(0.5689655172413793, 33)",[0.78873239 0.73239437 0.85714286 0.81428571 0...
4,"(0, 4, 6, 9, 10)",0.992366,0.878378,"(0.5689655172413793, 33)",[0.8028169 0.78873239 0.87142857 0.8 0...
5,"(0, 3, 6, 9, 10, 13)",0.978102,0.905405,"(0.5862068965517241, 34)",[0.78873239 0.8028169 0.78571429 0.8 0...
6,"(4, 6, 7, 8, 9, 13)",0.979021,0.945946,"(0.5862068965517241, 34)",[0.74647887 0.8028169 0.84285714 0.8 0...
7,"(0, 7, 9, 11, 12, 13)",0.985185,0.898649,"(0.5862068965517241, 34)",[0.77464789 0.81690141 0.85714286 0.82857143 0...
8,"(0, 6, 8, 10, 13, 14)",0.985612,0.925676,"(0.5689655172413793, 33)",[0.81690141 0.77464789 0.78571429 0.78571429 0...
9,"(0, 2, 5, 7, 9, 13)",0.978873,0.939189,"(0.5689655172413793, 33)",[0.8028169 0.8028169 0.84285714 0.78571429 0...


# same in the Classification

In [27]:

def get_selected_rows(columncombation,X_data):
    flag = False
    for i in columncombation:
        if(flag == False):
            c = X_data[:,i].T
            flag = True
        else:
            c = np.vstack((c,X_data[:,i].T))
    X_train_selected = c.T 
    return X_train_selected
def predicted_bugNum_vs_true_bugNum(Y_test_predict,Y_test_bugs):
    n = 0
    bugrowcount = 0
    for i in range(0,len(Y_test_predict)):
        if(Y_test_bugs[i]!=False):
            bugrowcount += 1
            if(Y_test_predict[i]==Y_test_bugs[i]):
                n=n+1
#    print(n/bugrowcount,n,bugrowcount)
    return n/bugrowcount,n
def load_one_version_method_matrix(matrix_path):
    print(matrix_path)
    return pd.read_csv(matrix_path)

def classification_binary(column_selected):

    filePath = "AlltheMatrix/"
    single_version_file_path = filePath + 'modified_single_version-ck-oo.csv'
    changed_file_path = filePath + 'modified_change_metrics.csv'
    bug_matrix_path = filePath + 'modified_bug-metrics.csv' 
    # complexity_code_change_path = 



    filePathNowusing = changed_file_path



    changed_matrix = load_one_version_method_matrix(filePathNowusing)
    print(f'lens: {len(changed_matrix.columns.values)}')

    startindex = 0
    endindex = 0
    for i, column in enumerate(changed_matrix.columns.values):
        if(column == 'classname'):
            startindex = i
        elif (column == 'bugs'):
            endindex = i
    print(f'start:{startindex},end:{endindex}') 
    target = changed_matrix['bugs'].values
    data = changed_matrix.values[:,startindex+1:endindex]
    print(target.shape)
    print(data.shape)

    #shuffleIndex
    import numpy as np
    shuffle_index = np.random.permutation(997)
    target, data = target[shuffle_index], data[shuffle_index]

    #shuffleIndex
    import numpy as np
    shuffle_index = np.random.permutation(997)
    target, data = target[shuffle_index], data[shuffle_index]

    #split the train set and the test set
    X_train = data[:700]
    X_test = data[700:]
    Y_train = target[:700]
    Y_test = target[700:]
    print(X_train,Y_train.shape)
    #train a binary classifier
    Y_train_bugs = (Y_train > 0)
    Y_test_bugs = (Y_test > 0 )

    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import cross_val_score

    from sklearn.naive_bayes import GaussianNB

    #hardCopy to test the selected rows
    columncombation_j = column_selected#(0, 6, 8, 10, 13, 14)
    X_train_selected = get_selected_rows(columncombation_j,X_train)
    X_test_selected = get_selected_rows(columncombation_j,X_test)


#     forest_clf = RandomForestClassifier(n_estimators = 10)
    
    forest_clf = GaussianNB()

    forest_clf.fit(X_train_selected,Y_train_bugs)

    cross_result = cross_val_score(forest_clf, X_train_selected, Y_train_bugs, cv = 10, scoring = 'accuracy')
    recall_result = cross_val_score(forest_clf, X_train, Y_train_bugs, cv = 5, scoring = 'recall')
    Y_forest_predict = forest_clf.predict(X_train_selected)

    from sklearn.metrics import precision_score, recall_score
    forst_precision = precision_score(Y_train_bugs, Y_forest_predict)
    forst_recall = recall_score(Y_train_bugs, Y_forest_predict)
    print(f'Forest_accurancy is {forst_precision}')
    print(f'Forest_recall is {forst_recall}')

    print(columncombation_j)
    #    print(X_train_selected)
    Y_test_predict = forest_clf.predict(X_test_selected)

    findingRate = predicted_bugNum_vs_true_bugNum(Y_test_predict,Y_test_bugs)
    print(f'fingdingrate = {findingRate}')
    return cross_result,recall_result

In [28]:
import pandas as pd
import os
import numpy as np

a = []
b = []
for i in range(20):
    a +=[ classification_binary((
        8, 9, 11, 13))[0]]
    b += [classification_binary((0, 1, 2, 3, 7, 8, 9, 11, 13))[1]]

AlltheMatrix/modified_change_metrics.csv
lens: 23
start:1,end:17
(997,)
(997, 15)
[[18 4 0 ... 0.44444399999999995 367.0 55.9708]
 [4 1 0 ... 25.5 164.71400000000003 18.6144]
 [18 4 0 ... 30.4444 169.857 73.1803]
 ...
 [709 69 0 ... 6.07334 367.0 100.595]
 [35 6 0 ... -0.685714 367.0 57.4762]
 [15 4 0 ... -0.866667 322.714 65.2597]] (700,)
Forest_accurancy is 0.6463414634146342
Forest_recall is 0.37857142857142856
(0, 1, 2, 3, 7, 8, 9, 11, 13)
fingdingrate = (0.36363636363636365, 24)
AlltheMatrix/modified_change_metrics.csv
lens: 23
start:1,end:17
(997,)
(997, 15)
[[41 17 1 ... 8.902439999999999 222.71400000000003 71.7756]
 [10 2 0 ... 2.4 105.0 22.3239]
 [80 20 0 ... -21.8125 367.0 81.4591]
 ...
 [24 8 0 ... 24.4167 238.429 63.1014]
 [68 16 0 ... 6.2794099999999995 367.0 129.77700000000002]
 [7 2 0 ... 0.0 231.571 59.6231]] (700,)
Forest_accurancy is 0.5949367088607594
Forest_recall is 0.34814814814814815
(0, 1, 2, 3, 7, 8, 9, 11, 13)
fingdingrate = (0.30985915492957744, 22)
AlltheMat

lens: 23
start:1,end:17
(997,)
(997, 15)
[[9 2 0 ... 40.8889 126.0 23.0066]
 [131 20 0 ... 1.41221 367.0 79.1748]
 [30 9 0 ... 2.86667 333.14300000000003 72.9204]
 ...
 [52 3 0 ... 6.88462 367.0 136.59]
 [46 7 0 ... 1.8913 239.143 65.4832]
 [23 7 0 ... 3.5217400000000003 332.0 76.0494]] (700,)
Forest_accurancy is 0.6746987951807228
Forest_recall is 0.3835616438356164
(0, 1, 2, 3, 7, 8, 9, 11, 13)
fingdingrate = (0.36666666666666664, 22)
AlltheMatrix/modified_change_metrics.csv
lens: 23
start:1,end:17
(997,)
(997, 15)
[[44 6 0 ... 1.15909 367.0 68.76]
 [49 6 0 ... -0.10204099999999999 367.0 57.4475]
 [13 5 0 ... 0.538462 345.42900000000003 47.2707]
 ...
 [9 4 0 ... -0.44444399999999995 256.57099999999997 48.3193]
 [2 0 0 ... 7.0 9.14286 1.2857100000000001]
 [2 0 0 ... -2.5 14.5714 5.0]] (700,)
Forest_accurancy is 0.6231884057971014
Forest_recall is 0.3282442748091603
(0, 1, 2, 3, 7, 8, 9, 11, 13)
fingdingrate = (0.36, 27)
AlltheMatrix/modified_change_metrics.csv
lens: 23
start:1,end:17


Forest_accurancy is 0.6551724137931034
Forest_recall is 0.4014084507042254
(0, 1, 2, 3, 7, 8, 9, 11, 13)
fingdingrate = (0.328125, 21)
AlltheMatrix/modified_change_metrics.csv
lens: 23
start:1,end:17
(997,)
(997, 15)
[[41 6 0 ... -0.21951199999999998 367.0 16.8509]
 [34 5 0 ... 1.14706 367.0 78.5451]
 [16 5 0 ... 1.0625 325.143 102.57600000000001]
 ...
 [46 5 0 ... 7.869560000000001 234.571 30.6146]
 [88 29 0 ... 5.04545 261.57099999999997 109.425]
 [81 18 0 ... 3.28395 367.0 130.299]] (700,)
Forest_accurancy is 0.6153846153846154
Forest_recall is 0.3310344827586207
(0, 1, 2, 3, 7, 8, 9, 11, 13)
fingdingrate = (0.3770491803278688, 23)
AlltheMatrix/modified_change_metrics.csv
lens: 23
start:1,end:17
(997,)
(997, 15)
[[7 4 0 ... 0.0 248.571 67.7302]
 [252 142 0 ... 9.19841 261.57099999999997 48.5992]
 [35 7 0 ... 121.4 333.14300000000003 105.181]
 ...
 [42 8 0 ... 1.33333 367.0 97.4341]
 [105 17 0 ... 13.1143 230.143 50.3423]
 [156 12 0 ... 6.788460000000001 367.0 152.001]] (700,)
Forest

In [20]:
sum(sum(a)/20)/10,sum(sum(b)/20)/5

(0.8253387717610007, 0.3834999986418229)

In [11]:
a

[array([0.8028169 , 0.81690141, 0.83098592, 0.8028169 , 0.87323944,
        0.8115942 , 0.89855072, 0.82608696, 0.88405797, 0.82608696]),
 array([0.81690141, 0.84285714, 0.77142857, 0.85714286, 0.82857143,
        0.78571429, 0.88571429, 0.91428571, 0.85714286, 0.7826087 ]),
 array([0.7       , 0.9       , 0.85714286, 0.77142857, 0.88571429,
        0.8       , 0.81428571, 0.81428571, 0.8       , 0.74285714]),
 array([0.77464789, 0.8028169 , 0.8028169 , 0.84285714, 0.82857143,
        0.88571429, 0.81428571, 0.84057971, 0.8115942 , 0.76811594]),
 array([0.88732394, 0.82857143, 0.85714286, 0.71428571, 0.9       ,
        0.84285714, 0.8       , 0.87142857, 0.81428571, 0.82608696]),
 array([0.77464789, 0.84507042, 0.78873239, 0.8028169 , 0.78571429,
        0.8       , 0.8115942 , 0.82608696, 0.76811594, 0.84057971]),
 array([0.87323944, 0.77464789, 0.76056338, 0.85714286, 0.95714286,
        0.75714286, 0.85714286, 0.79710145, 0.84057971, 0.84057971]),
 array([0.81690141, 0.83098592, 0.

In [25]:
import pandas as pd
import os
import numpy as np

a = []
b = []
for i in range(20):
    a +=[ classification_binary((0, 1, 2, 3, 7, 8, 9, 11, 13))[0]]
    b += [classification_binary((0, 1, 2, 3, 7, 8, 9, 11, 13))[1]]

AlltheMatrix/modified_change_metrics.csv
lens: 23
start:1,end:17
(997,)
(997, 15)
[[90 6 0 ... 10.4667 367.0 140.613]
 [20 6 0 ... 0.75 347.85699999999997 63.5369]
 [38 3 0 ... 4.947369999999999 248.571 71.0465]
 ...
 [40 4 0 ... 1.1 329.0 68.1264]
 [26 4 0 ... -0.0384615 367.0 35.0812]
 [11 4 0 ... 16.4545 222.857 13.1984]] (700,)
Forest_accurancy is 0.5
Forest_recall is 0.0273972602739726
(0, 1, 2, 3, 7, 8, 9, 11, 13)
fingdingrate = (0.0, 0)
AlltheMatrix/modified_change_metrics.csv
lens: 23
start:1,end:17
(997,)
(997, 15)
[[131 7 0 ... -0.381679 367.0 119.775]
 [48 5 0 ... 6.02083 367.0 81.3798]
 [93 17 0 ... -0.11828 367.0 54.9702]
 ...
 [198 13 0 ... 2.78283 367.0 105.595]
 [4 1 0 ... 25.5 164.71400000000003 18.6144]
 [45 11 0 ... 2.75556 367.0 122.181]] (700,)
Forest_accurancy is 0.5714285714285714
Forest_recall is 0.029411764705882353
(0, 1, 2, 3, 7, 8, 9, 11, 13)
fingdingrate = (0.0, 0)
AlltheMatrix/modified_change_metrics.csv
lens: 23
start:1,end:17
(997,)
(997, 15)
[[291 15 0 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarnin

Forest_accurancy is 0.0
Forest_recall is 0.0
(0, 1, 2, 3, 7, 8, 9, 11, 13)
fingdingrate = (0.0, 0)
AlltheMatrix/modified_change_metrics.csv
lens: 23
start:1,end:17
(997,)
(997, 15)
[[10 0 0 ... -3.1 89.0 2.20756]
 [7 0 0 ... 3.0 9.57143 3.71581]
 [86 10 0 ... -4.37209 367.0 85.1222]
 ...
 [15 5 0 ... -25.3333 220.143 34.5775]
 [36 13 0 ... 3.9722199999999996 333.14300000000003 72.3289]
 [172 70 0 ... 21.4651 264.857 166.729]] (700,)
Forest_accurancy is 0.0
Forest_recall is 0.0
(0, 1, 2, 3, 7, 8, 9, 11, 13)
fingdingrate = (0.0, 0)
AlltheMatrix/modified_change_metrics.csv
lens: 23
start:1,end:17
(997,)
(997, 15)
[[96 11 0 ... 2.10417 367.0 115.101]
 [31 4 0 ... 6.645160000000001 227.571 84.1364]
 [40 15 0 ... 3.625 333.14300000000003 69.6068]
 ...
 [1 0 0 ... 0.0 87.1429 0.0]
 [22 4 0 ... 7.68182 236.0 17.9238]
 [35 9 0 ... 0.48571400000000003 367.0 99.738]] (700,)
Forest_accurancy is 0.6
Forest_recall is 0.020134228187919462
(0, 1, 2, 3, 7, 8, 9, 11, 13)
fingdingrate = (0.01754385964912

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarnin

Forest_accurancy is 0.5714285714285714
Forest_recall is 0.026143790849673203
(0, 1, 2, 3, 7, 8, 9, 11, 13)
fingdingrate = (0.0, 0)
AlltheMatrix/modified_change_metrics.csv
lens: 23
start:1,end:17
(997,)
(997, 15)
[[35 7 0 ... 121.4 333.14300000000003 105.181]
 [66 6 0 ... 23.2727 329.14300000000003 99.6814]
 [52 10 0 ... 0.0 367.0 51.1952]
 ...
 [3 1 0 ... 0.333333 49.8571 19.1429]
 [56 7 0 ... 3.4464300000000003 367.0 50.3331]
 [36 13 0 ... 5.16667 333.14300000000003 72.5286]] (700,)
Forest_accurancy is 0.0
Forest_recall is 0.0
(0, 1, 2, 3, 7, 8, 9, 11, 13)
fingdingrate = (0.0, 0)
AlltheMatrix/modified_change_metrics.csv
lens: 23
start:1,end:17
(997,)
(997, 15)
[[30 4 0 ... 0.23333299999999998 367.0 39.4787]
 [10 3 0 ... 13.2 240.857 6.00676]
 [1 0 0 ... 0.0 14.7143 0.0]
 ...
 [10 3 0 ... 13.8 240.857 5.47462]
 [128 12 0 ... 15.7344 367.0 164.919]
 [29 20 0 ... -39.8621 126.571 59.8261]] (700,)
Forest_accurancy is 0.0
Forest_recall is 0.0
(0, 1, 2, 3, 7, 8, 9, 11, 13)
fingdingrate = (

...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


lens: 23
start:1,end:17
(997,)
(997, 15)
[[35 7 0 ... -0.257143 367.0 63.5052]
 [18 9 1 ... 8.77778 222.71400000000003 21.6087]
 [45 17 0 ... 8.82222 281.714 67.46]
 ...
 [42 15 0 ... 5.92857 333.14300000000003 61.1658]
 [193 30 0 ... 2.58549 367.0 111.711]
 [6 2 0 ... 4.33333 205.857 51.9851]] (700,)
Forest_accurancy is 0.0
Forest_recall is 0.0
(0, 1, 2, 3, 7, 8, 9, 11, 13)
fingdingrate = (0.0, 0)
AlltheMatrix/modified_change_metrics.csv
lens: 23
start:1,end:17
(997,)
(997, 15)
[[46 5 0 ... 7.869560000000001 234.571 30.6146]
 [4 4 0 ... 0.0 194.0 41.1429]
 [33 3 0 ... 1.39394 282.0 54.4366]
 ...
 [47 6 0 ... -0.42553199999999997 367.0 42.4421]
 [5 1 0 ... 2.2 179.857 14.205]
 [81 7 0 ... 2.6419799999999998 367.0 120.512]] (700,)
Forest_accurancy is 1.0
Forest_recall is 0.026143790849673203
(0, 1, 2, 3, 7, 8, 9, 11, 13)
fingdingrate = (0.0, 0)
AlltheMatrix/modified_change_metrics.csv
lens: 23
start:1,end:17
(997,)
(997, 15)
[[25 4 0 ... -0.44 367.0 36.6702]
 [49 4 0 ... 8.8367300000000

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


lens: 23
start:1,end:17
(997,)
(997, 15)
[[120 31 0 ... 10.925 333.14300000000003 131.366]
 [31 4 0 ... -0.16129000000000002 367.0 52.1911]
 [82 7 0 ... -1.9024400000000001 367.0 77.5985]
 ...
 [11 4 0 ... 18.6364 322.714 69.5672]
 [11 4 0 ... 1.09091 322.714 29.8517]
 [3 3 0 ... 0.0 168.857 56.0]] (700,)
Forest_accurancy is 0.0
Forest_recall is 0.0
(0, 1, 2, 3, 7, 8, 9, 11, 13)
fingdingrate = (0.0, 0)
AlltheMatrix/modified_change_metrics.csv
lens: 23
start:1,end:17
(997,)
(997, 15)
[[11 4 0 ... 1.09091 322.714 29.8517]
 [165 15 0 ... 2.49697 367.0 88.61399999999999]
 [1 0 0 ... 0.0 33.8571 0.0]
 ...
 [105 17 0 ... 13.1143 230.143 50.3423]
 [6 2 0 ... 6.83333 178.143 48.6098]
 [44 6 0 ... 0.454545 367.0 76.8418]] (700,)
Forest_accurancy is 0.0
Forest_recall is 0.0
(0, 1, 2, 3, 7, 8, 9, 11, 13)
fingdingrate = (0.0, 0)
AlltheMatrix/modified_change_metrics.csv
lens: 23
start:1,end:17
(997,)
(997, 15)
[[150 14 0 ... 1.42667 367.0 141.094]
 [38 13 0 ... 2.5526299999999997 333.14300000000003

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [29]:
sum(sum(a)/20)/10,sum(sum(b)/20)/5

(0.8273721896596975, 0.35917798729923317)